<a href="https://colab.research.google.com/github/Dimateos12/GraWZycie/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mnożenie Wektor Macierz


In [4]:
%%sh
cat > pi-mpi.c << EOF
#include <stdlib.h>
#include <stdio.h>
#include <mpi.h>
#include <time.h>
#include <sys/time.h>
#define N 4

MPI_Status status;
double A[N][N];
double B[N], C[N];

int main(int argc, char **argv) {
int processCount, processId, slaveTaskCount, source, dest, rows, offset;

struct timeval;

//Inicjalizuje uruchomienie środowiska MPI

MPI_Init(&argc, &argv);

//Odczytuje numer rank procesu wywołującego tę funkcje w obrębie danego komunikatora. 

MPI_Comm_rank(MPI_COMM_WORLD, &processId);

//Odczytuje ilość procesów w grupie skojarzonych z komunikatorem
MPI_Comm_size(MPI_COMM_WORLD, &processCount);

slaveTaskCount = processCount - 1;

//Proces główny 
if (processId == 0) 
{

srand ( time(NULL) );
for (int i = 0; i<N; i++) {
  for (int j = 0; j<N; j++) {
    A[i][j]= rand()%10;
  }
  B[i]= rand()%10;
}
printf("\nMacierz: \n");
for (int i = 0; i<N; i++) {
  for (int j = 0; j<N; j++) {
    printf("%.0f\t", A[i][j]);
  }
  printf("\n");
}
printf("\nWektor: \n");
for (int i = 0; i<N; i++) {
  printf("%.0f ", B[i]);
}

rows = N/slaveTaskCount;

offset = 0;

for (dest=1; dest <= slaveTaskCount; dest++)
{
  MPI_Send(&offset, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);
  MPI_Send(&rows, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);
  MPI_Send(&A[offset][0], rows*N, MPI_DOUBLE,dest,1, MPI_COMM_WORLD);
  MPI_Send(&B, N, MPI_DOUBLE, dest, 1, MPI_COMM_WORLD);
  offset = offset + rows;
}

for (int i = 1; i <= slaveTaskCount; i++)
{
  source = i;
  MPI_Recv(&offset, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
  MPI_Recv(&rows, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
  MPI_Recv(&C[offset], rows*N, MPI_DOUBLE, source, 2, MPI_COMM_WORLD, &status);
}

printf("\n\nWynikowy wektor: \n");
for (int i = 0; i<N; i++) {
  printf("%.0f ", C[i]);
}


}

//Proces do oczytu 

if (processId > 0) {
  source = 0;
    MPI_Recv(&offset, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);
  
  MPI_Recv(&rows, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);
  MPI_Recv(&A, rows*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);
  MPI_Recv(&B, N*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);
  for (int i = 0; i<rows; i++) {

    for (int j = 0; j<N; j++)
      C[i] = C[i] + A[i][j] * B[j];
  }

  MPI_Send(&offset, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);
  MPI_Send(&rows, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);
  MPI_Send(&C, rows*N, MPI_DOUBLE, 0, 2, MPI_COMM_WORLD);
}
MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 4 --allow-run-as-root a.out


Macierz: 
7	1	7	8	
3	1	7	7	
8	1	3	6	
4	0	4	2	

Wektor: 
6 2 2 3 

Wynikowy wektor: 
82 55 74 0 

# GRA O ŻYCIE 

In [ ]:
%%sh
cat > pi-mpi.c << EOF

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <string.h>
#include "mpi.h"

#define DEFAULT_ITERATIONS 64
#define GRID_WIDTH 256
#define DIM 16 // assume a square grid

int mod(int a, int b)
{
    int r = a % b;
    return r < 0 ? r + b : r;
}

int main(int argc, char **argv)
{
   // Tworzenie planszy do gry 

    int global_grid[256] =
        {0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0};

    // standardowe zmienne mpi 
    
    int num_procs;
    int ID, j;
    int iters = 0;
    int num_iterations;

    // Setup number of iterations
    if (argc == 1)
    {
        num_iterations = DEFAULT_ITERATIONS;
    }
    else if (argc == 2)
    {
        num_iterations = atoi(argv[1]);
    }
    else
    {
        printf("Usage: ./gameoflife <num_iterations>\n");
        exit(1);
    }

    // Messaging variables
    MPI_Status stat;

    // MPI Setup
    if (MPI_Init(&argc, &argv) != MPI_SUCCESS)
    {
        printf("MPI_Init error\n");
    }

    MPI_Comm_size(MPI_COMM_WORLD, &num_procs); // Set the num_procs
    MPI_Comm_rank(MPI_COMM_WORLD, &ID);

    assert(DIM % num_procs == 0);

    // Setup environment
    int *arr = (int *)malloc(DIM * ((DIM / num_procs) + 2) * sizeof(int));
    for (iters = 0; iters < num_iterations; iters++)
    {
        //printf("%d %d\n",ID, DIM * ((DIM / num_procs) + 2));
        j = DIM;
        for (int i = ID * (GRID_WIDTH / num_procs); i < (ID + 1) * (GRID_WIDTH / num_procs); i++)
        {
            arr[j] = global_grid[i];
            // if(ID==1)
            //     printf(" %d %d \n",j,i);
            j++;
        }

        if (num_procs != 1)
        {
            //odd-even send_recv
            int incoming_1[DIM];
            int incoming_2[DIM];
            int send_1[DIM];
            int send_2[DIM];
            if (ID % 2 == 0)
            {

                //first16
                for (int i = 0; i < DIM; i++)
                {
                    send_1[i] = arr[i + DIM];
                    // printf(" - %d - ",send_1[i]);
                    //printf(" %d %d\n ",i,i+DIM);
                }
                //first row to ID-1
                MPI_Ssend(&send_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD);

                //last16
                for (int i = 0; i < DIM; i++)
                {
                    send_2[i] = arr[(DIM * (DIM / num_procs)) + i];
                    // printf(" %d %d\n ",i,(DIM * (DIM / num_procs)) + i);
                }
                //last row to ID+1
                MPI_Ssend(&send_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD);
            }
            else
            {
                MPI_Recv(&incoming_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD, &stat);
                MPI_Recv(&incoming_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD, &stat);
            }
            if (ID % 2 == 0)
            {
                MPI_Recv(&incoming_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD, &stat);
                MPI_Recv(&incoming_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD, &stat);
            }
            else
            {
                //first16
                for (int i = 0; i < DIM; i++)
                {
                    send_1[i] = arr[i + DIM];
                }
                MPI_Ssend(&send_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD);

                //last16
                for (int i = 0; i < DIM; i++)
                {
                    send_2[i] = arr[(DIM * (DIM / num_procs)) + i];
                }
                MPI_Ssend(&send_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD);
            }
            for (int i = 0; i < DIM; i++)
            {
                arr[i] = incoming_1[i];
                arr[(DIM * ((DIM / num_procs) + 1)) + i] = incoming_2[i];
            }
        }
        else
        {
            for (int i = 0; i < DIM; i++)
            {
                arr[i + GRID_WIDTH + DIM] = global_grid[i];
                //printf(" %d %d \n",i + GRID_WIDTH+DIM,i);
            }
            for (int i = GRID_WIDTH; i < GRID_WIDTH + DIM; i++)
            {
                arr[i - GRID_WIDTH] = global_grid[i - DIM];
                //printf(" %d %d \n",i - GRID_WIDTH,i-DIM);
            }
        }
        //game logic neighbours
        int * final = (int *)malloc(DIM * ((DIM / num_procs)) * sizeof(int));

        for (int k = DIM; k < DIM * ((DIM / num_procs) + 1); k++)
        {
            int total_rows = DIM * (DIM / num_procs) + 2;
            int r = k / DIM;
            int c = k % DIM;
            int prev_r = mod(r - 1, total_rows);
            int prev_c = mod(c - 1, DIM);
            int next_r = mod(r + 1, total_rows);
            int next_c = mod(c + 1, DIM);

            int count = arr[prev_r * DIM + prev_c] + arr[prev_r * DIM + c] + arr[prev_r * DIM + next_c] + arr[r * DIM + prev_c] + arr[r * DIM + next_c] + arr[next_r * DIM + prev_c] + arr[next_r * DIM + c] + arr[next_r * DIM + next_c];
            if (arr[k] == 1)
            {
                if (count < 2)
                    final[k - DIM] = 0;
                else if (count > 3)
                    final[k - DIM] = 0;
                else
                    final[k - DIM] = 1;
            }
            else
            {
                if (count == 3)
                    final[k - DIM] = 1;
                else
                    final[k - DIM] = 0;
            }
        }

        j = 0;
        for (int i = ID * (GRID_WIDTH / num_procs); i < (ID + 1) * (GRID_WIDTH / num_procs); i++)
        {
            global_grid[i] = final[j];
            j++;
        }
        MPI_Gather(final, DIM * (DIM / num_procs), MPI_INT, &global_grid, DIM * (DIM / num_procs), MPI_INT, 0, MPI_COMM_WORLD);

        // Output the updated grid state
        if (ID == 0)
        {
            printf("\nIteration %d: final grid:\n", iters);
            for (j = 0; j < GRID_WIDTH; j++)
            {
                if (j % DIM == 0)
                {
                    printf("\n");
                }
                printf("%d  ", global_grid[j]);
            }
            printf("\n");
        }
    }

    // Clean up memory
    free(arr);
    MPI_Finalize(); // finalize so I can exit
}

EOF
mpicc pi-mpi.c && mpirun -n 4 --allow-run-as-root a.out

# Całka 


In [7]:
%%sh
cat > pi-mpi.c << EOF

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <string.h>
#include "mpi.h" 
#include <math.h>    

double wzorfunkcji(double x)
{
  return pow(x, 2);
}

double wys(double a, double b, double N)
{
  return (b - a) / N;
}

int main(int argc,char **argv)
{
  int rank;
  int num_proc;
  int tag = 5;
  MPI_Status status;

  MPI_Init(&argc,&argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &num_proc);

  // Metoda Trapezów
  double a = 2;
  double b = 4;
  double xi = 0;
  double suma = 0;

  double start = MPI_Wtime();

  if(rank == num_proc - 1)
  { 
    xi += (wzorfunkcji(a + (((double) rank / (double) num_proc) * (b - a))));
    MPI_Send(&xi,1,MPI_DOUBLE,rank-1,tag,MPI_COMM_WORLD);

  }
  else if(rank != 0 && rank < num_proc - 1)
  { 
    MPI_Recv(&xi,1,MPI_DOUBLE,rank+1,tag,MPI_COMM_WORLD,&status);
    xi += (wzorfunkcji(a + (((double) rank / (double) num_proc) * (b - a))));           
    MPI_Send(&xi,1,MPI_DOUBLE,rank-1,tag,MPI_COMM_WORLD);
  }
  else
  {
    MPI_Recv(&xi,1,MPI_DOUBLE,rank+1,tag,MPI_COMM_WORLD,&status);
    suma = wys(a,b,num_proc) * (wzorfunkcji(a) / 2 + xi + wzorfunkcji(b)/2);
    double end = MPI_Wtime();
    printf("Wynik całki numerycznej metodą trapezów = %f \n", suma);
    printf("Czas obliczen %f",end - start);
  }

  MPI_Finalize();
  return 0;
}

EOF
mpicc pi-mpi.c -lm && mpirun -n 4 --allow-run-as-root a.out

Wynik całki numerycznej metodą trapezów = 18.750000 
Czas obliczen 0.000019

# Pi - Leibniz

In [8]:
%%sh
cat > zadanie1.c << EOF
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <string.h>
#include "mpi.h" 
#include <math.h>    
int main(int argc, char *argv[])
{
  long max_iter = 100000;
  double suma = 0.0;
  double suma_fin;

  int rank, size;
  MPI_Init( &argc, &argv );
  MPI_Comm_size( MPI_COMM_WORLD, &size );
  MPI_Comm_rank( MPI_COMM_WORLD, &rank );


  for (long i = rank; i< max_iter; i+=size){
    double x = 1.0 / (2.0 * (double)i + 1);
    if (i % 2 == 0){
      suma += x;
      } else { 
        suma -= x;
    }
  }
 
  MPI_Reduce(&suma, &suma_fin, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
  if (rank == 0){
    suma_fin = suma_fin * 4.0;
    printf("PI wynosi %f", suma_fin);
  }
  MPI_Finalize();
  return 0;
}
EOF
mpicc zadanie1.c && mpirun -n 4 --allow-run-as-root a.out
     

PI wynosi 3.141583